# Model South Korea Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.data_plots_kr import load_df_feat_kr
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

from my_helpers.model_kr import prepare_data_features_kr
from my_helpers.model_kr import prepare_dataset_kr

from my_helpers.utils import clean_file

import settings

In [2]:
tf.__version__

'2.3.0'

### Definitions

In [3]:
PATH_TO_SAVE_DATA = settings.PATH_TO_SAVE_DATA

from my_helpers.model_kr import PATH_MDL_MULTI_STEP_KR
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_KR 
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_FILE_KR
from my_helpers.model_kr import PATH_SERVERLESS_KR
from my_helpers.model_kr import URL_PREDICT_KR
from my_helpers.model_kr import PAST_HISTORY # days used to predict next values in future
from my_helpers.model_kr import FUTURE_TARGET  # predict 3 days later
from my_helpers.model_kr import STEP 

from my_helpers.data_plots_kr import PATH_DF_FEAT_KR

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model_kr import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")


TRAIN_SPLIT = 383


### helper functions

### load data

In [4]:
# load
df_feat_kr = load_df_feat_kr()
# clean
df_feat_kr = prepare_data_features_kr(df_feat_kr)
# split train/test
df_feat_kr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_kr.shape[0])]
df_feat_kr

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2020-04-03,10062.0,424365.0,174.0,2020-04-03,5.0,0.0,1.0,5.0,1.0,1.0,...,5.000000,10.0,inf,106.0,4.240000,20.0,20.000000,5.0,inf,True
2020-04-04,10156.0,434888.0,177.0,2020-04-04,6.0,0.0,1.0,1.0,0.0,0.0,...,5.000000,10.0,inf,67.0,1.030769,21.0,21.000000,5.0,inf,True
2020-04-05,10237.0,441662.0,183.0,2020-04-05,0.0,3.0,1.0,4.0,0.0,1.0,...,6.000000,9.0,9.000000,60.0,0.789474,21.0,10.500000,8.0,inf,True
2020-04-06,10284.0,447509.0,186.0,2020-04-06,1.0,0.0,2.0,2.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.769231,22.0,7.333333,8.0,inf,True
2020-04-07,10331.0,456654.0,192.0,2020-04-07,2.0,0.0,1.0,1.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.759494,22.0,5.500000,8.0,inf,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-30,122007.0,8765781.0,1828.0,2021-04-30,5.0,1.0,37.0,29.0,5.0,21.0,...,0.498328,57.0,0.703704,344.0,1.522124,615.0,2.043189,36.0,0.900000,False
2021-05-01,122634.0,8798176.0,1831.0,2021-05-01,6.0,7.0,43.0,12.0,2.0,10.0,...,0.484746,53.0,0.616279,327.0,1.373950,620.0,1.993569,42.0,1.050000,False
2021-05-02,123240.0,8811627.0,1833.0,2021-05-02,0.0,1.0,61.0,30.0,1.0,12.0,...,0.458904,52.0,0.597701,329.0,1.316000,614.0,1.686813,42.0,1.076923,False


In [5]:
df_feat_kr[df_feat_kr["train"] == False].head(5)

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2021-04-22,116661.0,8470366.0,1808.0,2021-04-22,4.0,4.0,58.0,23.0,8.0,6.0,...,1.088496,78.0,2.166667,264.0,1.466667,472.0,1.794677,43.0,1.592593,False
2021-04-23,117458.0,8513591.0,1811.0,2021-04-23,5.0,3.0,63.0,22.0,6.0,9.0,...,0.951020,73.0,1.622222,279.0,1.603448,520.0,2.157676,44.0,1.692308,False
2021-04-24,118243.0,8554654.0,1812.0,2021-04-24,6.0,3.0,52.0,32.0,1.0,9.0,...,0.832700,67.0,1.288462,285.0,1.507937,526.0,1.962687,40.0,1.290323,False
2021-04-25,118887.0,8577231.0,1813.0,2021-04-25,0.0,3.0,45.0,24.0,9.0,7.0,...,0.624161,70.0,1.228070,296.0,1.517949,555.0,2.047970,36.0,0.947368,False
2021-04-26,119386.0,8594894.0,1817.0,2021-04-26,1.0,1.0,29.0,25.0,0.0,6.0,...,0.535032,67.0,1.135593,306.0,1.478261,567.0,2.115672,34.0,0.850000,False


### Prepare dataset

In [6]:
dataset, data_std, data_mean = prepare_dataset_kr(df_feat_kr)
print(dataset)
print("data_mean : ", data_mean)
print("data_std : ", data_std)

[[-0.43550547 -0.02694187 -1.0880055  ... -0.62770085  0.99902656
  -0.37290234]
 [-0.43212007 -0.11983649 -1.25851332 ... -0.62672801  1.49788859
  -0.37290234]
 [-0.69956656 -0.48398342 -1.7681605  ... -0.85467525 -1.49528356
  -0.37290234]
 ...
 [-0.0800386  -0.01207873 -0.20216225 ... -0.44869934 -1.49528356
   0.17858476]
 [-0.25946473  0.32234192 -0.3041855  ... -0.37415591 -0.99642154
  -0.11459922]
 [ 0.28219906 -0.08267864  0.66538718 ...  1.9410681  -0.49755951
  -0.11459922]]
data_mean :  [9.52140992e+00 1.90750218e+01 6.21032202e+01 1.85167102e+01
 2.74723238e+02 2.08306632e+04 2.99738903e+00 4.41071316e+01]
data_std :  [9.84620612e+00 8.97073797e+00 1.42124467e+01 6.12912827e+00
 3.16381923e+02 1.64467886e+04 2.00456228e+00 5.92371312e+00]


In [7]:
dataset.shape[1]

8

In [8]:
PAST_HISTORY

14

In [9]:
dataset.shape[1]

8

### Load model

In [10]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP_KR)

CPU times: user 2.42 s, sys: 171 ms, total: 2.59 s
Wall time: 2.94 s


In [11]:
multi_step_model.inputs[0].dtype

tf.float32

In [13]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_lstm"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/keras_lstm/assets


### Save model TFlite

In [14]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE_KR)

File /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model_20210511_15_05_47.tflite moved!


In [15]:
open(PATH_MDL_MULTI_TFLITE_FILE_KR, "wb").write(tflite_model)

5500

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [16]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[320 - 334]
[327 - 341]
[334 - 348]
[341 - 355]
[348 - 362]
[355 - 369]
[362 - 376]
[369 - 383]
[376 - 390]
9


array([[387.82007, 411.9751 , 430.24475, 439.15607, 441.3852 , 437.75293,
        439.79294, 424.04526, 428.22974, 430.51733, 432.63434, 432.86108,
        428.59088, 428.1844 , 460.5235 , 478.85693, 489.43683, 489.61658,
        482.63538, 475.30728, 472.8897 , 448.33148, 464.00012, 472.22394,
        472.36066, 465.88385, 461.7871 , 461.26175, 506.49487, 511.87335,
        512.8555 , 512.31903, 505.591  , 503.0622 , 501.0896 , 544.46594,
        552.7407 , 555.1989 , 554.0486 , 546.1938 , 541.2438 , 538.40924,
        627.0459 , 632.6356 , 630.8204 , 626.328  , 615.32556, 610.3817 ,
        607.37024, 630.21423, 649.6271 , 657.74915, 652.0238 , 634.5901 ,
        623.003  , 615.7429 , 654.47375, 670.02765, 675.46375, 669.5513 ,
        652.5901 , 641.9247 , 634.3913 ]], dtype=float32)

#### Future days

In [17]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [18]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [19]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE_KR)

In [20]:
PATH_MDL_MULTI_TFLITE_FILE_KR

'/Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model.tflite'

### Predict reloaded model

#### Past days

In [21]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [22]:
x_multi.shape

(1, 14, 8)

In [23]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [24]:
dataset.shape

(397, 8)

In [25]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[320 - 334]
[327 - 341]
[334 - 348]
[341 - 355]
[348 - 362]
[355 - 369]
[362 - 376]
[369 - 383]
[376 - 390]


In [26]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda_kr.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [27]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [28]:
y_multi_pred

array([[0.35746935, 0.43381694, 0.49156258, 0.5197289 , 0.52677464,
        0.515294  , 0.52174187, 0.4719676 , 0.48519358, 0.49242413,
        0.49911538, 0.49983218, 0.4863352 , 0.48505032, 0.58726573,
        0.6452128 , 0.6786532 , 0.67922133, 0.6571556 , 0.6339934 ,
        0.62635213, 0.54872996, 0.5982544 , 0.62424767, 0.62467986,
        0.60420835, 0.59125966, 0.58959913, 0.73256916, 0.74956906,
        0.75267357, 0.7509778 , 0.7297123 , 0.7217194 , 0.7154845 ,
        0.8525857 , 0.87874   , 0.88650984, 0.8828738 , 0.85804695,
        0.8424012 , 0.83344203, 1.1135993 , 1.1312668 , 1.1255293 ,
        1.1113302 , 1.0765544 , 1.0609282 , 1.0514095 , 1.1236134 ,
        1.1849723 , 1.210644  , 1.1925477 , 1.1374444 , 1.1008207 ,
        1.0778733 , 1.2002915 , 1.2494532 , 1.2666353 , 1.2479476 ,
        1.1943376 , 1.160627  , 1.136816  ]], dtype=float32)

In [29]:
y_multi_pred_out

array([[0.35746935, 0.43381694, 0.4915626 , 0.5197289 , 0.52677464,
        0.51529396, 0.52174187, 0.47196758, 0.48519352, 0.49242413,
        0.49911538, 0.49983218, 0.48633516, 0.48505032, 0.58726579,
        0.64521283, 0.67865318, 0.67922139, 0.65715557, 0.63399351,
        0.62635219, 0.54872996, 0.59825432, 0.62424761, 0.62467992,
        0.60420835, 0.59125966, 0.58959913, 0.73256922, 0.74956912,
        0.75267363, 0.75097775, 0.72971237, 0.72171944, 0.71548456,
        0.85258555, 0.87873995, 0.88650972, 0.88287377, 0.85804695,
        0.84240115, 0.83344191, 1.1135993 , 1.13126695, 1.12552941,
        1.11133027, 1.07655454, 1.06092823, 1.0514096 , 1.12361324,
        1.18497252, 1.21064413, 1.1925478 , 1.13744438, 1.10082078,
        1.07787335, 1.20029151, 1.24945331, 1.26663542, 1.24794769,
        1.19433773, 1.16062725, 1.13681626]])

In [30]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [31]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [32]:
y_future_pred_out

array([[1.02913344, 1.10524964, 1.14445782, 1.13605237, 1.0905273 ,
        1.0564965 , 1.03692591]])

In [33]:
y_future_pred

array([[1.0291334, 1.1052498, 1.1444578, 1.1360524, 1.0905273, 1.0564965,
        1.0369258]], dtype=float32)

In [34]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda-kr
    
- Execute : sls deploy -v

In [50]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE_KR}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr\nserverless deploy -v'

In [51]:
open('deploy_serverless_kr.sh', "w").write(str_exe)
os.chmod('deploy_serverless_kr.sh', stat.S_IRWXU)

In [52]:
!cat ./deploy_serverless_kr.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr
serverless deploy -v

In [54]:
!./deploy_serverless_kr.sh

Serverless: Deprecation warning: Resolution of lambda version hashes was improved with better algorithm, which will be used in next major release.
            Switch to it now by setting "provider.lambdaHashingVersion" to "20201221"
            More Info: https://www.serverless.com/framework/docs/deprecations/#LAMBDA_HASHING_VERSION_V2
Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/ef4e42eb03bbad46f74fee99a3c01f994e8119e7a557f947779093ab37b248d3_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverle

### API AWS real Test

#### Past days

In [55]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[320 - 334]
[327 - 341]
[334 - 348]
[341 - 355]
[348 - 362]
[355 - 369]
[362 - 376]
[369 - 383]
[376 - 390]
status code :  200
[[[0.3574693202972412, 0.43381690979003906, 0.4915626049041748, 0.5197288990020752, 0.5267746448516846, 0.5152939558029175, 0.5217418670654297]], [[0.4719676375389099, 0.48519352078437805, 0.4924241304397583, 0.49911537766456604, 0.4998321831226349, 0.4863351881504059, 0.4850503206253052]], [[0.5872657299041748, 0.6452127695083618, 0.6786531805992126, 0.6792213320732117, 0.6571555733680725, 0.63399338722229, 0.626352071762085]], [[0.5487300157546997, 0.598254382610321, 0.624247670173645, 0.6246799230575562, 0.6042083501815796, 0.5912597179412842, 0.5895991325378418]], [[0.73256915807724, 0.7495690584182739, 0.7526736259460449, 0.7509777545928955, 0.7297123074531555, 0.7217193841934204, 0.7154844999313354]], [[0.8525856137275696, 0.8787400126457214, 0.8865097165107727, 0.8828737735748291, 0.8580468893051147, 0.8424011468887329, 0.833441972732544]], [[1.113599300

In [56]:
len(json_list_list_x)

21254

In [57]:
resp.json()

[[[0.3574693202972412,
   0.43381690979003906,
   0.4915626049041748,
   0.5197288990020752,
   0.5267746448516846,
   0.5152939558029175,
   0.5217418670654297]],
 [[0.4719676375389099,
   0.48519352078437805,
   0.4924241304397583,
   0.49911537766456604,
   0.4998321831226349,
   0.4863351881504059,
   0.4850503206253052]],
 [[0.5872657299041748,
   0.6452127695083618,
   0.6786531805992126,
   0.6792213320732117,
   0.6571555733680725,
   0.63399338722229,
   0.626352071762085]],
 [[0.5487300157546997,
   0.598254382610321,
   0.624247670173645,
   0.6246799230575562,
   0.6042083501815796,
   0.5912597179412842,
   0.5895991325378418]],
 [[0.73256915807724,
   0.7495690584182739,
   0.7526736259460449,
   0.7509777545928955,
   0.7297123074531555,
   0.7217193841934204,
   0.7154844999313354]],
 [[0.8525856137275696,
   0.8787400126457214,
   0.8865097165107727,
   0.8828737735748291,
   0.8580468893051147,
   0.8424011468887329,
   0.833441972732544]],
 [[1.1135993003845215,
   1

In [58]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [59]:
y_multi_pred_out

array([[0.35746932, 0.43381691, 0.4915626 , 0.5197289 , 0.52677464,
        0.51529396, 0.52174187, 0.47196764, 0.48519352, 0.49242413,
        0.49911538, 0.49983218, 0.48633519, 0.48505032, 0.58726573,
        0.64521277, 0.67865318, 0.67922133, 0.65715557, 0.63399339,
        0.62635207, 0.54873002, 0.59825438, 0.62424767, 0.62467992,
        0.60420835, 0.59125972, 0.58959913, 0.73256916, 0.74956906,
        0.75267363, 0.75097775, 0.72971231, 0.72171938, 0.7154845 ,
        0.85258561, 0.87874001, 0.88650972, 0.88287377, 0.85804689,
        0.84240115, 0.83344197, 1.1135993 , 1.13126695, 1.12552941,
        1.11133027, 1.07655454, 1.06092823, 1.0514096 , 1.12361324,
        1.18497241, 1.21064413, 1.1925478 , 1.13744438, 1.10082066,
        1.07787335, 1.2002914 , 1.24945307, 1.2666353 , 1.24794769,
        1.19433749, 1.16062713, 1.13681614]])

In [60]:
y_multi_pred

array([[0.35746935, 0.43381694, 0.49156258, 0.5197289 , 0.52677464,
        0.515294  , 0.52174187, 0.4719676 , 0.48519358, 0.49242413,
        0.49911538, 0.49983218, 0.4863352 , 0.48505032, 0.58726573,
        0.6452128 , 0.6786532 , 0.67922133, 0.6571556 , 0.6339934 ,
        0.62635213, 0.54872996, 0.5982544 , 0.62424767, 0.62467986,
        0.60420835, 0.59125966, 0.58959913, 0.73256916, 0.74956906,
        0.75267357, 0.7509778 , 0.7297123 , 0.7217194 , 0.7154845 ,
        0.8525857 , 0.87874   , 0.88650984, 0.8828738 , 0.85804695,
        0.8424012 , 0.83344203, 1.1135993 , 1.1312668 , 1.1255293 ,
        1.1113302 , 1.0765544 , 1.0609282 , 1.0514095 , 1.1236134 ,
        1.1849723 , 1.210644  , 1.1925477 , 1.1374444 , 1.1008207 ,
        1.0778733 , 1.2002915 , 1.2494532 , 1.2666353 , 1.2479476 ,
        1.1943376 , 1.160627  , 1.136816  ]], dtype=float32)

In [61]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [62]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[1.0291334390640259, 1.1052496433258057, 1.1444578170776367, 1.1360523700714111, 1.0905272960662842, 1.056496500968933, 1.0369259119033813]]]


In [63]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[1.02913344, 1.10524964, 1.14445782, 1.13605237, 1.0905273 ,
        1.0564965 , 1.03692591]])

In [64]:
y_future_pred

array([[1.0291334, 1.1052498, 1.1444578, 1.1360524, 1.0905273, 1.0564965,
        1.0369258]], dtype=float32)

In [65]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
